## Reporting: wragle_report
* Create a **300-600 word written report** called "wrangle_report.pdf" or "wrangle_report.html" that briefly describes your wrangling efforts. This is to be framed as an internal document.

The data gathering section can be summarized to 2 main parts: downloading the data using requests and querying the missing information using tweepy API. 

The programmatic download of data was done by providing the URL of the files `twitter_archive_enhanced.csv` and `image-predictions.tsv`. After downloading the two files, they were read into the dataframes, `df_twitter_archive`and `df_prediction` respectively. 

The query on the other hand required the making of the developer account to get access to twitter API and using bearer token to get access for the script, ing the tweepy package. According to Twitter API documentation, the limit is 100 queries per request, and 900 requests per 15 minutes. No rate limiting was necessary. To make the queries, we needed to look up the tweets by id, which were available in the `df_prediction` data frame. Since there were over 2000 tweets to query, a list of lists containing twitter id's grouped in 100s was created and saved in `tweet_ids` variable. This list was passed to the client module `client.get_tweets(ids, tweet_fields='public_metrics')` and requested to retrieve `public_metrics` which contains the `like_count` and `retweet_count`. 

For the sake of completeness, the file `tweet_json.txt` was saved within the same loop that grabbed the `like_count` and `retweet_count`. The final dataframe containing the latst mentioned information was saved into a variable `df_tweets`, which had 2057 entries. Some entries were missing  since they were removed or no longer availble. This was discovered by finding the missing `tweet_id` and querying them again, this time trying to get the error message since data field was empty.

In the assessing section, first look was using `df_twitter_archive.info()` too look at the data type mismatches and if there were any missing values. tweet_id was in int but it's significance was not of numerical value so it needed to be converted to string. timestamp was in string format and it should be converted to datetime format. Next, the name column was inspected via `df_twitter_archive.name.value_counts()`, and it showed that there are 745 None values but they were in string format, besides few other entries that were invalid ( such as 'a','just' etc). Then, we wanted to check if there are any entries that had more than 100 as `rating_numerator`, a few were found and it would seem that these entries relate to multiple dogs in a single photo. In other instances, it was relating to a punchline or an attempt to make a meme (such case with dog named `Atticus` with rating `1776/10`).

Before proceeding to clean the data, a summary of 11 issues (9 quality, 2 tidiness) was noted down. A copy of the original data was made and saved into variables `df_archive_clean`, `df_prediction_clean` and `df_tweet_clean`. The issues are listed below:

### Quality issues
1. `timestamp` in `df_twitter_archive` column are in object/string format.
> This was fixed by using `pd.to_datetime(df_archive_clean['timestamp'])`

2. Columns `name` till `puppo` in `df_twitter_archive` have `None` values but are in string format.
> This was fixed by creating a list with the column names, and looping over the list to apply the function `(lambda x: None if x == 'None' else x)`

3. `tweet_id` column in `df_twitter_archive` is int format need to change to str.
> This was achieved by `df_archive_clean.tweet_id =  df_archive_clean.tweet_id.astype(str)`

4. Some name entries in `df_twitter_archive.name` are invalid (`'a'`, `'just'`, etc). 
> This was fixed by applying lambda function to the `name` column: `(lambda x: x if x != None and x[0] == x[0].upper() else None)`

5. Rating numerator in `df_twitter_archive` has invalid inputs along with decimals not extracted correctly, need to sue text to re-extract
> This was fixed by specifying a function `re_extract` to re-extract the ratings from text and considering decimals as well.

6. Denominator of tweets in `df_twitter_archive` are inconsistent (according to WeRateDogs wikipedia page, it should always be out of 10). Need to exclude tweets with invalid rates
> This was fixed by querying `df_archive_clean.query('rating_denominator%10 == 0 and rating_denominator != 0')`

7. `tweet_id` in `df_tweet` is in int format, need to change to string format.
> This was fixed by querying `df_archive_clean.query(not expanded_urls.isna())`

8. There are entries missing from `df_tweet` when compared to the `df_twitter_archive` dataframe, only ids that exist in both shall be maintained.
> This was fixed by placing the tweet ids availbe in `df_tweet` in a variable `tweet_ids` and querying `df_archive_clean.query('tweet_id in @tweet_ids')`

9. rows that have `retweet_id` should be removed since they are not original tweets. Also, rows that don't have a valid rating should be removed.

> To remove the retweet rows: `query('retweeted_status.isna()')`
> To remove the rows with invalid ratings: `query('not rating.isna()')`


### Tidiness issues
1. Columns `doggo` till `puppo` are actually one variable `stage`, need to be combined into one column stage.
> This was performed by summing columns and applying the function `lambda x: x if x != None else '')`

2. Columns relating to retweet and in_reply are not needed and must be removed.
> This was performed by selecting the related columns `df_archive_clean[['tweet_id','timestamp','text','rating_numerator','rating_denominator','stage']]`, also the rows containing retweets were removed by querying `query(retweet_status_id.isna())`
2. Rating numerator and denominator are actually one variable `rating`, need to be combined into one column `rating`, and consider the floating number to be the scale of each photo regardless of number of dogs in the photo.
> This was fixed by simple including the needed columns and overwriting the `rating` column with the division of `rating_numerator` and `rating_denominator`

Lastly, the data was stored by making a copy of the `df_archive_clean` and saving it into the variable `df_master_archive`. The columns `like_count` and `retweet_count` were added to the `df_master_archive` from `df_tweet_clean`, and then df_prediction was merged with the master data frame on `tweet_id`, finally the data was saved to `twitter_archive_master.csv`.